Pandas (.csv)

In [1]:
import pandas as pd
import io

# Simulando um arquivo CSV (normalmente você leria com pd.read_csv('arquivo.csv'))
csv_data = """id;texto_review
1;O produto é excelente e chegou rápido!
2;Não gostei, a caixa veio amassada e o produto sujo.
3;Comprei para minha mãe, ela adorou as cores.
4;Péssimo atendimento, nunca mais compro.
5;O celular é bom, mas a bateria dura pouco.
"""

# Carregando para o Pandas
# O separador ';' é comum em arquivos brasileiros
df = pd.read_csv(io.StringIO(csv_data), sep=';')

# Vamos ver como o Pandas enxerga isso
print(df.head())

   id                                       texto_review
0   1             O produto é excelente e chegou rápido!
1   2  Não gostei, a caixa veio amassada e o produto ...
2   3       Comprei para minha mãe, ela adorou as cores.
3   4            Péssimo atendimento, nunca mais compro.
4   5         O celular é bom, mas a bateria dura pouco.


pd:

    Vem da linha: import pandas as pd.

    O que é: É um apelido (alias). Poderíamos ter chamado de qualquer coisa (como import pandas as abacaxi), mas a comunidade mundial de Python combinou de usar pd para não ter que digitar a palavra "pandas" inteira toda vez que chamar uma função.

df:

    Vem de DataFrame.

    O que é: É o nome da estrutura de dados principal do Pandas (aquela tabela com linhas e colunas).

    Como é muito comum trabalhar com DataFrames, os programadores abreviam a variável para df para ser rápido de escrever.

Pergunta guia: Ao rodar esse código, você consegue identificar qual coluna representa o nosso Documento (unidade básica de texto) segundo a definição do seu PDF? Me diga o nome dela para avançarmos para a parte do NLTK/SpaCy

A coluna texto_review é onde estão nossos dados brutos. Para o Pandas, é apenas uma string (texto), mas para nós é a fonte de informação.

NLTK -> Tokenização / Token

In [2]:
import nltk
from nltk.tokenize import word_tokenize

# Baixando o recurso necessário para o NLTK saber o que é pontuação
nltk.download('punkt_tab') 

frase = "O celular é bom, mas a bateria dura pouco."

# Tokenizando (note que especificamos o idioma)
tokens = word_tokenize(frase, language='portuguese')

print(tokens)
print(f"Quantidade: {len(tokens)}")

['O', 'celular', 'é', 'bom', ',', 'mas', 'a', 'bateria', 'dura', 'pouco', '.']
Quantidade: 11


[nltk_data] Downloading package punkt_tab to /home/matheus-
[nltk_data]     sales/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
from nltk.tokenize import sent_tokenize

texto_longo = "O celular é bom. A bateria dura pouco."

# Tokenização por sentenças
tokens_sentenca = sent_tokenize(texto_longo, language='portuguese')

print(tokens_sentenca)
print(f"Quantidade: {len(tokens_sentenca)}")

['O celular é bom.', 'A bateria dura pouco.']
Quantidade: 2


word_tokenize (Tokenização por Palavras):

    O que faz: Quebra o texto nas menores unidades de significado.

    Resultado: Palavras individuais e pontuações.

    Quando usar: Quando você precisa analisar o vocabulário, contar frequência de palavras ou analisar a gramática (saber o que é verbo, substantivo, etc.).

    Analogia: É como desmontar a parede de Lego tijolo por tijolo.

sent_tokenize (Tokenização por Sentenças):

    O que faz: Quebra o texto em ideias completas (frases). Ele procura por "pistas" de fim de frase, como ponto final, exclamação ou interrogação.

    Resultado: Frases inteiras.

    Quando usar: Quando você quer analisar o sentimento de uma frase inteira ou resumir um texto parágrafo por parágrafo.

    Analogia: É como separar a parede de Lego em grandes blocos ou seções, sem desmontar os tijolinhos ainda.

SpaCy -> Processamento e Análise Gramatical

Baixar o modelo de língua portuguesa do SpaCy

In [4]:
import spacy

# Normalmente precisamos baixar o modelo antes no terminal com: 
# python -m spacy download pt_core_news_sm
# Mas no Jupyter, podemos tentar carregar direto se já estiver instalado:

try:
    nlp = spacy.load("pt_core_news_sm")
    print("Modelo carregado com sucesso!")
except OSError:
    print("Precisamos baixar o modelo. Tente rodar no terminal: python -m spacy download pt_core_news_sm")

Modelo carregado com sucesso!


In [18]:
# Pegando a frase do índice 4 do nosso DataFrame
texto_exemplo = df['texto_review'][3]

# A mágica acontece aqui: O SpaCy processa tudo e guarda no objeto 'doc'
doc = nlp(texto_exemplo)

# Vamos ver a classificação (POS - Part of Speech) de cada token
print(f"Frase: {texto_exemplo}\n")
print(f"{'TOKEN':<15} {'POS (Classe)':<15} {'EXPLICAÇÃO'}")
print("-" * 45)

for token in doc:
    # token.text é a palavra
    # token.pos_ é a etiqueta gramatical (POS Tag)
    print(f"{token.text:<15} {token.pos_:<15} {spacy.explain(token.pos_)}")

Frase: Péssimo atendimento, nunca mais compro.

TOKEN           POS (Classe)    EXPLICAÇÃO
---------------------------------------------
Péssimo         ADJ             adjective
atendimento     NOUN            noun
,               PUNCT           punctuation
nunca           ADV             adverb
mais            ADV             adverb
compro          ADJ             adjective
.               PUNCT           punctuation


Testes revelaram exatamente por que PLN é uma área tão desafiadora (e divertida). Vamos analisar o "raciocínio" da máquina nesses erros:

    "Comprei" virou PROPN (Nome Próprio):

        PROPN é a etiqueta para nomes como "Maria", "Brasil" ou "Google".

        Por que o erro? Como a palavra "Comprei" estava no início da frase, ela tinha letra maiúscula. O modelo sm (small/pequeno) do SpaCy é otimizado para ser leve e rápido, então ele usa atalhos estatísticos. Ele viu a maiúscula e "chutou" que era um nome, em vez de analisar a conjugação verbal.

    "Compro" virou ADJ (Adjetivo):

        Por que o erro? A frase "nunca mais compro" tem uma estrutura de negação. O modelo se confundiu com o contexto e achou que "compro" estava qualificando algo, em vez de ser a ação.

A Lição de Ouro aqui: Modelos de PLN são probabilísticos, não exatos. O modelo pt_core_news_sm acerta muito, mas desliza em ambiguidades. Em projetos reais profissionais, costumamos usar o modelo lg (large/grande), que é bem mais pesado (centenas de MBs), mas entende essas nuances muito melhor.

1. Os Atributos do SpaCy (.text e .pos_)

Esses são atributos nativos dos objetos Token do SpaCy. Quando o nlp processa o texto, ele cria esses objetos e já preenche essas informações para você.

    token.text: É a palavra original, do jeito que estava no texto (ex: "celular").

    token.pos_: É a etiqueta da classe gramatical (ex: "NOUN", "ADJ").

        Curiosidade: Notou esse sublinhado _ no final? O SpaCy faz isso porque ele guarda internamente as informações como números (para economizar memória). O _ diz: "me mostre a versão legível em texto (string) desse código numérico".

2. A Formatação do Python (:<15)

Essa parte (:<15) não é do SpaCy, é um recurso do próprio Python chamado f-string formatting. Serve apenas para deixar o visual bonito e organizado, como uma tabela.

Dentro das chaves { }, os dois pontos : avisam que vamos começar uma regra de formatação.

    < (Menor que): Significa alinhar à esquerda (o texto encosta na esquerda).

    15: Significa reservar 15 espaços para esse texto.

Visualmente, acontece isso: Imagine que a palavra é "bom" (3 letras).

    Sem formatação: "bom"

    Com :<15: "bom " (ele preenche com espaços vazios até dar 15 caracteres).

Isso garante que a próxima coluna comece sempre na mesma posição, criando aquele visual de tabela alinhada.



In [13]:
# 1. Criamos uma função que recebe um texto e devolve só os adjetivos
def extrair_adjetivos(texto):
    doc = nlp(texto)
    # Lista com o texto de cada token, MAS SÓ SE o POS for "ADJ"
    adjetivos = [token.text for token in doc if token.pos_ == "ADJ"]
    return adjetivos

# 2. Aplicamos essa função na coluna inteira e criamos uma NOVA coluna
# df['nome_da_nova_coluna'] = df['coluna_original'].apply(funcao)
df['adjetivos_encontrados'] = df['texto_review'].apply(extrair_adjetivos)

# Mostramos o resultado
print(df[['texto_review', 'adjetivos_encontrados']])

                                        texto_review adjetivos_encontrados
0             O produto é excelente e chegou rápido!           [excelente]
1  Não gostei, a caixa veio amassada e o produto ...                    []
2       Comprei para minha mãe, ela adorou as cores.                    []
3            Péssimo atendimento, nunca mais compro.     [Péssimo, compro]
4         O celular é bom, mas a bateria dura pouco.                 [bom]


Joblib -> Persistência / salvar

In [21]:
import joblib

# Definindo o nome do arquivo
nome_arquivo = "dataset_processado.pkl"

# Salvando (dump) o objeto 'df' inteiro no disco
joblib.dump(df, nome_arquivo)

print(f"Arquivo salvo como: {nome_arquivo}")

Arquivo salvo como: dataset_processado.pkl


A última etapa do seu desenho é: Nova Coluna ──(Joblib)──> Arquivo Final

Em Ciência de Dados, muitas vezes o processamento demora horas. Imagine ter que rodar todo esse nlp() de novo só porque fechou o notebook? O Joblib serve para "congelar" seus dados processados e salvá-los no disco, mantendo toda a estrutura (listas, dicionários, tipos de variáveis) intacta.

Joblib -> Persistência / carregar

In [22]:
import joblib

# Carregando os dados do disco de volta para uma variável
# Note que eu dei um nome novo (df_recuperado) só para provar que é um novo objeto
df_recuperado = joblib.load('dataset_processado.pkl')

# Vamos ver se a nossa coluna de adjetivos sobreviveu?
print(df_recuperado[['texto_review', 'adjetivos_encontrados']].head())

                                        texto_review adjetivos_encontrados
0             O produto é excelente e chegou rápido!           [excelente]
1  Não gostei, a caixa veio amassada e o produto ...                    []
2       Comprei para minha mãe, ela adorou as cores.                    []
3            Péssimo atendimento, nunca mais compro.     [Péssimo, compro]
4         O celular é bom, mas a bateria dura pouco.                 [bom]


Em computação, esses pares são clássicos: save/load, dump/load, push/pull. O joblib segue a tradição do Python (igual à biblioteca pickle) e usa load para ler o arquivo e reconstruir seu objeto na memória.